In [100]:
import requests
import csv

In [136]:
def cleantitle (title):
    title = title.replace("?", "").replace("*", "").replace("\n", " ").strip()
    return title

doicollection = []
surveyedtitles = []

with open("Literature 12e01.csv", "r") as f:
    d = csv.DictReader(f)
    for item in d:
        surveyedtitles.append(cleantitle(item["\ufeffName"]))
        if (item["DOI"] == ""): 
            # print(item)
            continue
        doi = item["DOI"]
        doicollection.append(doi)

print(surveyedtitles)

['Layout of Compound Directed Graphs', 'BaryLayout: A Simple Algorithm for Fast Drawing Large Graphs', 'An experimental comparison of four graph drawing algorithms', 'Fast filtering and animation of large dynamic networks', 'A New Approach to Exact Crossing Minimization', 'An ILP-based Proof System for the Crossing Number Problem', 'Visualizing the evolution of compound digraphs with TimeArcTrees', 'The State of the Art in Visualizing Dynamic Graphs', 'TimeArcs: Visualizing Fluctuations in Dynamic Networks', 'https://scholar.google.com/citationsview_op=view_citation&hl=en&user=VAi0mIQAAAAJ&citation_for_view=VAi0mIQAAAAJ:ufrVoPGSRksC', 'A Force-Directed Algorithm that Preserves Edge Crossing Properties', 'Fast and Simple Horizontal Coordinate Assignment', 'Applying Crossing Reduction Strategies to Layered Compound Graphs', 'An Experimental Study of Crossing Minimization Heuristics', 'A Fast and Simple Heuristic for Constrained Two-Level Crossing Reduction', 'An Experimental Comparison o

In [126]:
# do an api request for each doi in the list, add all citations to a list
def get_all_references(doicollection):
    reflist = []
    for doi in doicollection:
        if "https://" in doi or "http://" in doi:
                doi = doi.split("/")[3] + "/" + doi.split("/")[4]
        try: 
            api_url = "https://api.crossref.org/works/" + doi
            response = requests.get(api_url)
            try:
                reflist.append(response.json()['message']['reference'])
            except:
                print("No reference found for " + doi)
        except:
            print("Error: " + doi + " not found")

    return reflist

reflist = get_all_references(doicollection)

No reference found for 10.5555/647554.729577
No reference found for 10.5555/1964371.1964394
No reference found for 10.7155/jgaa.00442
No reference found for 10.7155/jgaa.00440
No reference found for 10.1109/TPDS.2018.2869805
Error: 10.1007/978-3-319-73915-1_31 not found
No reference found for 10.7155/jgaa.00467
No reference found for 10.7155/jgaa.00474
No reference found for 10.7155/jgaa.00487
No reference found for 10.1109/TVCG.2020.3038154
No reference found for 10.1109/tvcg.2020.3030442
No reference found for 10.1109/tvcg.2021.3114759
No reference found for 10.1109/TVCG.2021.3114795
No reference found for 10.1109/TVCG.2021.3114756


In [105]:
totalcount = 0
for item in reflist:
    # print(len(item))
    totalcount += len(item)
print(len(reflist), totalcount)

refdict = {}
for paper in reflist:
    for i, ref in enumerate(paper):
        # print(ref)
        if ref['key'] not in refdict:
            refdict[ref['key']] = {"count": 1}
        else:
            refdict[ref['key']]["count"] += 1
        if 'article-title' in ref:
            refdict[ref['key']]["title"] = ref['article-title']
        if 'author' in ref:
            refdict[ref['key']]["author"] = ref['author']
        if 'series-title' in ref:
            refdict[ref['key']]["series"] = ref['series-title']
        if 'unstructured' in ref:
            refdict[ref['key']]["unstructured"] = ref['unstructured']

        if 'series-title' in ref and ref['series-title'] == "Lect Notes Comput Sci":
            print(ref)
        # if i == 10: break
    # break

print(len(refdict.keys()))

74 1980
{'key': '3_CR1', 'series-title': 'Lect Notes Comput Sci', 'doi-asserted-by': 'publisher', 'first-page': '104', 'DOI': '10.1007/3-540-44969-8_5', 'volume-title': 'Drawing Graphs: Methods and Models', 'author': 'O. Bastert', 'year': '2001', 'unstructured': 'Oliver Bastert and Christian Matuszewski. Layered drawings of digraphs. In Michael Kaufmann and Dorothea Wagner, editors, Drawing Graphs: Methods and Models, volume 2025 of Lecture Notes in Computer Science, pages 104–139. Springer, 2001.'}
{'key': '3_CR3', 'series-title': 'Lect Notes Comput Sci', 'doi-asserted-by': 'crossref', 'first-page': '229', 'DOI': '10.1007/3-540-44541-2_22', 'volume-title': 'A fast layout algorithm for k-level graphs', 'author': 'C. Buchheim', 'year': '2001', 'unstructured': 'Christoph Buchheim, Michael Jünger, and Sebastian Leipert. A fast layout algorithm for k-level graphs. In Joe Marks, editor, Proceedings of the 8th International Symposium on Graph Drawing (GD 2000), volume 1984 of Lecture Notes i

In [142]:
reflist = []
for item in refdict:
    if "title" in refdict[item]:
        if cleantitle(refdict[item]["title"]) in surveyedtitles:
            continue
    else:
        if "unstructured" not in refdict[item]: 
            # print(refdict[item])
            continue
    
        beforepapertitle = refdict[item]["unstructured"].split(":")[0]
        afterpapertitle = refdict[item]["unstructured"][len(beforepapertitle) + 1:].split(".")[0]
        if afterpapertitle in surveyedtitles:
            continue
    
    if refdict[item]['count'] > 0:
        # print(refdict[item])
        # print(refdict[item])
        reflist.append(refdict[item])

    # toprint = str(refdict[item]['count'])
    # if 'title' in refdict[item]:
    #     toprint += ", " + refdict[item]['title']
    # if 'series' in refdict[item]:
    #     toprint += ", \t" + refdict[item]['series']
    # print(toprint)

reflist.sort(key=lambda x: x['count'], reverse=True)
output = open("references.csv", "w")
for item in reflist:
    if 'unstructured' in item:
        output.write(str(item['count']) + ", " + item['unstructured'] + "\n")
    else:
        output.write(str(item['count']) + ", " + item['title'] + "\n")


In [165]:
# note: this part updates a table on notion. it requires a file called "notion_key" in the same directory as this file, containing the API key for notion. It won't work unless the key is there.
NOTION_URL = 'https://api.notion.com/v1/databases/'
TABLE_ID = "d8cc9a7fcaaf4e6c92ed5b00ef4a3a81"
API_KEY = open("notion_key", "r").read()

database_url = NOTION_URL + TABLE_ID + ""
response = requests.get(database_url, headers={"Authorization": 'Bearer ' + API_KEY + '', 'Notion-Version': '2022-02-22'})
if response.status_code != 200:
    print("error connecting to notion, " + str(response), response.text, database_url)
else:
    print(response.json())

{'object': 'database', 'id': 'd8cc9a7f-caaf-4e6c-92ed-5b00ef4a3a81', 'cover': None, 'icon': {'type': 'emoji', 'emoji': '🪴'}, 'created_time': '2022-04-17T15:09:00.000Z', 'created_by': {'object': 'user', 'id': '18632548-4ae0-4a0a-a8c0-a6c4ee6b359c'}, 'last_edited_by': {'object': 'user', 'id': '18632548-4ae0-4a0a-a8c0-a6c4ee6b359c'}, 'last_edited_time': '2022-04-17T15:10:00.000Z', 'title': [{'type': 'text', 'text': {'content': 'papers_todo', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'papers_todo', 'href': None}], 'properties': {'Tags': {'id': 'z%7BWf', 'name': 'Tags', 'type': 'multi_select', 'multi_select': {'options': []}}, 'Name': {'id': 'title', 'name': 'Name', 'type': 'title', 'title': {}}}, 'parent': {'type': 'page_id', 'page_id': '0cc6b5e4-5430-4aec-98f3-b59b1a720476'}, 'url': 'https://www.notion.so/d8cc9a7fcaaf4e6c92ed5b00ef4a3a81', 'archived': False}


In [173]:
url = "https://api.notion.com/v1/databases/" + TABLE_ID

headers = {
    "Accept": "application/json",
    "Notion-Version": "2022-02-22",
    "Content-Type": "application/json",
    "Authorization": 'Bearer ' + API_KEY + ''
}

update_data = {
    "title": [{

            "text": {

                "content": "Today'\''s grocery list"

            }

        }]
}

response = requests.request("PATCH", url, headers=headers, json=update_data)

print(response.text)

{"object":"database","id":"d8cc9a7f-caaf-4e6c-92ed-5b00ef4a3a81","cover":null,"icon":{"type":"emoji","emoji":"🪴"},"created_time":"2022-04-17T15:09:00.000Z","created_by":{"object":"user","id":"18632548-4ae0-4a0a-a8c0-a6c4ee6b359c"},"last_edited_by":{"object":"user","id":"22259768-cb86-48f3-8a7c-65f04293c9cd"},"last_edited_time":"2022-04-18T18:22:00.000Z","title":[{"type":"text","text":{"content":"Today'''s grocery list","link":null},"annotations":{"bold":false,"italic":false,"strikethrough":false,"underline":false,"code":false,"color":"default"},"plain_text":"Today'''s grocery list","href":null}],"properties":{"Tags":{"id":"z%7BWf","name":"Tags","type":"multi_select","multi_select":{"options":[]}},"Name":{"id":"title","name":"Name","type":"title","title":{}}},"parent":{"type":"page_id","page_id":"0cc6b5e4-5430-4aec-98f3-b59b1a720476"},"url":"https://www.notion.so/d8cc9a7fcaaf4e6c92ed5b00ef4a3a81","archived":false}


In [186]:
def append_citation(refentry):
    # append a page to the table
    url = "https://api.notion.com/v1/pages"

    name = ""
    if "title" in refentry:
        name = refentry["title"]
    else:
        name = refentry["unstructured"]
    name = name.strip()

    payload = {
        "parent": { "database_id": TABLE_ID },
        "properties": {
            "Name": {
                "title": [
                    {
                        "text": {
                            "content": name
                        }
                    }
                ]
            },
            "Count": {
                "number": refentry['count']
            }
        },
        # "children": ["Unknown Type: mixed type"],
    }

    headers = {
        "Authorization": 'Bearer ' + API_KEY + '',
        "Accept": "application/json",
        "Notion-Version": "2022-02-22",
        "Content-Type": "application/json"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    print(response.text)

In [188]:
reflist.sort(key=lambda x: x['count'], reverse=True)
append_citation(reflist[0])
for item in reflist:
    append_citation(item)


{"object":"page","id":"6bc016b9-015d-42c1-b212-1ff0838c1b18","created_time":"2022-04-18T18:39:00.000Z","last_edited_time":"2022-04-18T18:39:00.000Z","created_by":{"object":"user","id":"22259768-cb86-48f3-8a7c-65f04293c9cd"},"last_edited_by":{"object":"user","id":"22259768-cb86-48f3-8a7c-65f04293c9cd"},"cover":null,"icon":null,"parent":{"type":"database_id","database_id":"d8cc9a7f-caaf-4e6c-92ed-5b00ef4a3a81"},"archived":false,"properties":{"Count":{"id":"z%7BWf","type":"number","number":6},"Name":{"id":"title","type":"title","title":[{"type":"text","text":{"content":"Ackerman, E., Tardos, G.: On the maximum number of edges in quasi-planar graphs. J. Comb. Theory, Ser. A 114(3), 563–571 (2007). https://doi.org/10.1016/j.jcta.2006.08.002","link":null},"annotations":{"bold":false,"italic":false,"strikethrough":false,"underline":false,"code":false,"color":"default"},"plain_text":"Ackerman, E., Tardos, G.: On the maximum number of edges in quasi-planar graphs. J. Comb. Theory, Ser. A 114(3)